# BioLLM x Plants - Idea Mining, Divergent-Convergent Script

Rachel K. Luu, Ming Dao, Subra Suresh, Markus J. Buehler (2025) [full reference to be updated to be included here]

## Divergent Generation
For the divergent generation phase, BioinspiredLLM quantized to 8bit is used.

In [ ]:
from llama_index.core import PromptTemplate, SimpleDirectoryReader, VectorStoreIndex, Settings
from llama_index.llms.llama_cpp import LlamaCPP
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.response.notebook_utils import display_response
from typing import List, Optional, Sequence
import pandas as pd
from utils.formats import messages_to_prompt, completion_to_prompt

url = "https://huggingface.co/rachelkluu/Llama3.1-8b-Instruct-CPT-SFT-DPO-09022024-Q8_0-GGUF/resolve/main/llama3.1-8b-instruct-cpt-sft-dpo-09022024-q8_0.gguf"
bioinspiredllm_q8 = LlamaCPP(
    model_url=url,
    model_path=None,
    temperature=1,
    max_new_tokens=2048,
    context_window=16000,
    model_kwargs={"n_gpu_layers": -1},
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=False,  
)

Settings.llm = bioinspiredllm_q8 
Settings.embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-en-v1.5"
)

documents = SimpleDirectoryReader(
    "./RAG/" 
).load_data()

Settings.chunk_size = 128
Settings.chunk_overlap = 50
vector_index = VectorStoreIndex.from_documents(documents)
query_engine = vector_index.as_query_engine(response_mode="compact", similarity_top_k=10) 

In [2]:
prompt = "What are other material applications that pollen grains can be used for?"
num_per_gen = "" #can optionally specify how many ideas per generation
sim_thres = 0.7 #adjusts the similarity threshold, with values closer to 1 being greater similarity 
num_ideas = 100 #number of ideas desired in total
csv_filename = "divergentideas"

In [3]:
from protocols.idea_div import sample_bullets, filter_unique_ideas

finaldf = pd.DataFrame({
    "Prompt":[],
    "Idea":[],
})

while len(finaldf) < num_ideas:
    gendf, bullets, bullcount = sample_bullets(1, num_per_gen, prompt,query_engine) 
    print(f"{bullcount} ideas were generated")
    finaldf = filter_unique_ideas(finaldf, gendf, similarity_threshold= sim_thres)
    print(f"Unique ideas were added. Current number of ideas: {len(finaldf)}")

finaldf.to_csv(f'{csv_filename}.csv', index=False)
print(finaldf)
print(f"The final list of ideas was generated and saved in {csv_filename}.csv.")

11 Ideas were generated
Unique rows were added. Current number of rows: 11
15 Ideas were generated
Unique rows were added. Current number of rows: 26
9 Ideas were generated
Unique rows were added. Current number of rows: 35
79 Ideas were generated
Unique rows were added. Current number of rows: 66
71 Ideas were generated
Unique rows were added. Current number of rows: 79
17 Ideas were generated
Unique rows were added. Current number of rows: 94
25 Ideas were generated
Unique rows were added. Current number of rows: 114
                                                Prompt  \
0    What are other material applications that poll...   
1    What are other material applications that poll...   
2    What are other material applications that poll...   
3    What are other material applications that poll...   
4    What are other material applications that poll...   
..                                                 ...   
109  What are other material applications that poll...   
110  What a

## Convergent Evaluation
For the convergent evaluation phase, Llama-3.1-8b-instruct quantized to 8bit is used. Considering the memory of your system, you may need to restart the kernel and load only one model at a time. 

In [ ]:
from llama_index.core import PromptTemplate, SimpleDirectoryReader, VectorStoreIndex, Settings
from llama_index.llms.llama_cpp import LlamaCPP
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.response.notebook_utils import display_response
from typing import List, Optional, Sequence
import pandas as pd
from utils.formats import messages_to_prompt, completion_to_prompt

url = "https://huggingface.co/rachelkluu/Meta-Llama-3.1-8B-Instruct-Q8_0-GGUF/resolve/main/meta-llama-3.1-8b-instruct-q8_0.gguf"
llama31_q8 = LlamaCPP(
    model_url=url,
    model_path=None,
    temperature=.1,
    max_new_tokens=5000,
    context_window=16000,
    model_kwargs={"n_gpu_layers": -1},
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=True,
)

Settings.llm = llama31_q8 
Settings.embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-en-v1.5"
)

documents = SimpleDirectoryReader(
    "./RAG/"   # FOLDER TO PAPERS OF INTEREST
).load_data()

Settings.chunk_size = 128
Settings.chunk_overlap = 50

vector_index = VectorStoreIndex.from_documents(documents)
query_engine = vector_index.as_query_engine(response_mode="compact", similarity_top_k=10) 

In [2]:
# original prompt from divergent generation: 
prompt = "What are other material applications that pollen grains can be used for?"
round_limit = 6 # Maximum number of elimination rounds to perform (for 100 ideas, 5-6 rounds)
filename=[f"divergentideas.csv",] # csv file where ideas from divergent generation was saved 
filetagline = "convergedideas" # tagline for final outputted csv files

In [3]:
from protocols.idea_con import pairwise_elim_and_rate

# Filename to store the elimination results and final winners
outputfilename = f'{filetagline}_elim.csv'
# Filename to store the ratings collected in the first round
ratings_outputfile = f'{filetagline}_rate.csv'

# Run the pairwise elimination and rating function
pairwise_elim_and_rate(filename, prompt, round_limit, outputfilename, ratings_outputfile,query_engine)

--- Round 1 ---



llama_print_timings:        load time =    1267.51 ms
llama_print_timings:      sample time =       4.15 ms /    43 runs   (    0.10 ms per token, 10351.47 tokens per second)
llama_print_timings: prompt eval time =    1913.81 ms /  1485 tokens (    1.29 ms per token,   775.94 tokens per second)
llama_print_timings:        eval time =    1382.03 ms /    42 runs   (   32.91 ms per token,    30.39 tokens per second)
llama_print_timings:       total time =    3407.53 ms /  1527 tokens
Llama.generate: 250 prefix-match hit, remaining 1129 prompt tokens to eval

llama_print_timings:        load time =    1267.51 ms
llama_print_timings:      sample time =       1.01 ms /    12 runs   (    0.08 ms per token, 11846.00 tokens per second)
llama_print_timings: prompt eval time =     738.25 ms /  1129 tokens (    0.65 ms per token,  1529.28 tokens per second)
llama_print_timings:        eval time =     364.96 ms /    11 runs   (   33.18 ms per token,    30.14 tokens per second)
llama_print_timings:

--- Round 2 ---



llama_print_timings:        load time =    1267.51 ms
llama_print_timings:      sample time =       1.56 ms /    12 runs   (    0.13 ms per token,  7677.54 tokens per second)
llama_print_timings: prompt eval time =     789.49 ms /  1187 tokens (    0.67 ms per token,  1503.50 tokens per second)
llama_print_timings:        eval time =     378.44 ms /    11 runs   (   34.40 ms per token,    29.07 tokens per second)
llama_print_timings:       total time =    1185.92 ms /  1198 tokens
Llama.generate: 71 prefix-match hit, remaining 1350 prompt tokens to eval

llama_print_timings:        load time =    1267.51 ms
llama_print_timings:      sample time =       1.08 ms /    12 runs   (    0.09 ms per token, 11131.73 tokens per second)
llama_print_timings: prompt eval time =     897.25 ms /  1350 tokens (    0.66 ms per token,  1504.60 tokens per second)
llama_print_timings:        eval time =     379.71 ms /    11 runs   (   34.52 ms per token,    28.97 tokens per second)
llama_print_timings: 

--- Round 3 ---


...

--- Round 5 ---



llama_print_timings:        load time =    1267.51 ms
llama_print_timings:      sample time =       1.67 ms /    12 runs   (    0.14 ms per token,  7189.93 tokens per second)
llama_print_timings: prompt eval time =     830.36 ms /  1259 tokens (    0.66 ms per token,  1516.20 tokens per second)
llama_print_timings:        eval time =     368.62 ms /    11 runs   (   33.51 ms per token,    29.84 tokens per second)
llama_print_timings:       total time =    1218.17 ms /  1270 tokens
Llama.generate: 71 prefix-match hit, remaining 1299 prompt tokens to eval

llama_print_timings:        load time =    1267.51 ms
llama_print_timings:      sample time =       1.42 ms /    12 runs   (    0.12 ms per token,  8438.82 tokens per second)
llama_print_timings: prompt eval time =     866.07 ms /  1299 tokens (    0.67 ms per token,  1499.88 tokens per second)
llama_print_timings:        eval time =     376.54 ms /    11 runs   (   34.23 ms per token,    29.21 tokens per second)
llama_print_timings: 

--- Round 6 ---



llama_print_timings:        load time =    1267.51 ms
llama_print_timings:      sample time =       1.35 ms /    12 runs   (    0.11 ms per token,  8915.30 tokens per second)
llama_print_timings: prompt eval time =     718.61 ms /  1086 tokens (    0.66 ms per token,  1511.25 tokens per second)
llama_print_timings:        eval time =     368.88 ms /    11 runs   (   33.53 ms per token,    29.82 tokens per second)
llama_print_timings:       total time =    1103.19 ms /  1097 tokens
Llama.generate: 250 prefix-match hit, remaining 1112 prompt tokens to eval

llama_print_timings:        load time =    1267.51 ms
llama_print_timings:      sample time =       0.91 ms /    12 runs   (    0.08 ms per token, 13129.10 tokens per second)
llama_print_timings: prompt eval time =     743.49 ms /  1112 tokens (    0.67 ms per token,  1495.65 tokens per second)
llama_print_timings:        eval time =     357.13 ms /    11 runs   (   32.47 ms per token,    30.80 tokens per second)
llama_print_timings:

Final round winners (Top 2 ideas): ['Pollen-based bioinks for 3D printing and biomedical applications', '**Biocompatible scaffolds:** Pollen-based scaffolds could be developed for tissue engineering applications, providing a more biocompatible and sustainable alternative to synthetic scaffolds.']
Results (including rounds) exported to convergedideas_elim.csv.
Ratings from the first round exported to convergedideas_rate.csv.
